## Pipelines Challenge

In this challenge, we will be working with this [dataset](https://drive.google.com/file/d/1B07fvYosBNdIwlZxSmxDfeAf9KaygX89/view?usp=sharing), where we will be predicting sales. 

**The main goal is to create a `pipeline` that covers all the data preprocessing and modeling steps.**


**TASK 1**: Build a pipeline that ends with a regression model, to predict `Item_Outlet_Sales` from the dataset. 

**The pipeline should have following steps:**

1. Split the features into numerical and categorical (text)
2. Replace null values
    - the mean for numerical variables
    - the most frequent value for categorical variables
3. Create dummy variables from categorical features
4. Use a PCA to reduce number of dummy variables to 3 principal components. PCA will be used directly after the OneHotEncoder that outputs data into a SparseMatrix, so we will need to use the **ToDenseTransformer** from the [article about custom pipelines](https://queirozf.com/entries/scikit-learn-pipelines-custom-pipelines-and-pandas-integration).
5. Select the 3 best candidates from the original numerical features using KBest
6. Fit a Ridge regression (default alpha is fine for now)

**TASK 2**: Tune the parameters of multiple models as well as the preprocessing steps and find the best solution.
- Try these models: 
        - Random Forest Regressor
        - Gradient Boosting Regressor 
        - Ridge Regression. 
- For the task 2, we will need to use the same approach from this [earlier article](https://iaml.it/blog/optimizing-sklearn-pipelines), in the section `PIPELINE TUNING (ADVANCED VERSION)`, where we tried different kinds of scalers. (Use the article as reference.)

_________________________________

In [1]:
import pandas as pd
df = pd.read_csv("regression_exercise.csv")
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [2]:
# creating target variable
y = df["Item_Outlet_Sales"]
df = df.drop(["Item_Outlet_Sales","Item_Identifier"],axis = 1)

Split the dataset into a train and test set.

**Note:** We should always do this at the beginning before the pipeline.

In [3]:
df_train = df.sample(frac=0.8).sort_index()
y_train = y[y.index.isin(df_train.index.tolist())]

In [4]:
df_test = df[~df.index.isin(df_train.index.tolist())].sort_index()
y_test = y[y.index.isin(df_test.index.tolist())]

---------------------
## Task I

### Split Features into numerical and categorical

In [5]:
cat_feats = df.dtypes[df.dtypes == 'object'].index.tolist()
num_feats = df.dtypes[~df.dtypes.index.isin(cat_feats)].index.tolist()

In [6]:
from sklearn.preprocessing import FunctionTransformer

# Using own function in Pipeline
def numFeat(data):
    return data[num_feats]

def catFeat(data):
    return data[cat_feats]

In [7]:
# we will start two separate pipelines for each type of features
keep_num = FunctionTransformer(numFeat)
keep_cat = FunctionTransformer(catFeat)

### replacing null values

In [8]:
# Use SimpleImputer
from sklearn.impute import SimpleImputer
import numpy as np

In [9]:
imp_mean = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imp_mode = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')

### Creating dummy variables

In [10]:
# use OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

In [11]:
enc = OneHotEncoder()

### Use PCA to reduce the number of dummy variables to 3 principal components.

In [12]:
# don't forget ToDenseTransformer after one hot encoder

In [13]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

class ToDenseTransformer():

    # here you define the operation it should perform
    def transform(self, X, y=None, **fit_params):
        return X.todense()

    # just return self
    def fit(self, X, y=None, **fit_params):
        return self


In [14]:
pca = PCA(n_components = 3)

### Select the 3 best numeric features

In [15]:
# use SelectKBest
from sklearn.feature_selection import SelectKBest

In [16]:
selection = SelectKBest(k=3)

### Fitting models

In [17]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Use base_model in Task I
base_model = Ridge()

### Building a Pipeline

In [18]:
from sklearn.pipeline import Pipeline, FeatureUnion

In [19]:
pipeline1 = Pipeline(steps = [('numFeats', keep_num),
                             ('missing_num',imp_mean),
                            ('selection',selection)])

pipeline2 = Pipeline(steps = [('numCats',keep_cat),
                             ('missing_cats',imp_mode),
                             ('ohe',enc),
                             ('to_dense',ToDenseTransformer()),
                             ('pca',pca)])

In [20]:
pipeline3 = FeatureUnion([("pipeline1", pipeline1), ("pipeline2", pipeline2)])

In [21]:
pipeline = Pipeline(steps = [('features',pipeline3),
                            ('ridge',base_model)])

In [22]:
# model.score(df_test,y_test)
pipeline.fit(df_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('pipeline1',
                                                 Pipeline(steps=[('numFeats',
                                                                  FunctionTransformer(func=<function numFeat at 0x0000028A3C7DAB80>)),
                                                                 ('missing_num',
                                                                  SimpleImputer()),
                                                                 ('selection',
                                                                  SelectKBest(k=3))])),
                                                ('pipeline2',
                                                 Pipeline(steps=[('numCats',
                                                                  FunctionTransformer(func=<function catFeat at 0x0000028A3E8A3940>)),
                                                                 ('missing_cats',
        

In [23]:
pipeline.score(df_test, y_test)

0.3988819422316492

----------------------------
## Task II

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [25]:
ridge = Ridge()
rfr = RandomForestRegressor()
gbr = GradientBoostingRegressor()

models = [ridge, rfr, gbr]

params = [{'features__pipeline1__selection__k':[3,4,5],
           'features__pipeline2__pca__n_components': [3,4,5],
           'models':[ridge],
           'models__alpha':[0.001,0.01,0.1,1,10]
          },
          {
        'features__pipeline1__selection__k':[3,4,5],
        'features__pipeline2__pca__n_components': [3, 4,5],
        'models':[rfr],
        "models__n_estimators":[100, 150, 200],
        "models__max_depth":[5,7, 10]
        
    },
    {
        'features__pipeline1__selection__k':[3,4,5],
        'features__pipeline2__pca__n_components': [3, 4,5],
        'models':[gbr],
        "models__learning_rate":[0.1, 0.2, 0.3],
        "models__n_estimators":[100, 150, 200],
        "models__max_depth":[5,7, 10]
    }]

final_pipeline = Pipeline(steps = [('features',pipeline3),('models',base_model)])


In [26]:
gridsearch = GridSearchCV(final_pipeline, params, verbose=5).fit(df_train, y_train)


Fitting 5 folds for each of 369 candidates, totalling 1845 fits
[CV 1/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=0.001;, score=0.416 total time=   0.1s
[CV 2/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=0.001;, score=0.366 total time=   0.2s
[CV 3/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=0.001;, score=0.378 total time=   0.1s
[CV 4/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=0.001;, score=0.379 total time=   0.1s
[CV 5/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=0.001;, score=0.376 total time=   0.1s
[CV 1/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=0.01;, sc

[CV 3/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=5, models=Ridge(), models__alpha=0.001;, score=0.380 total time=   0.1s
[CV 4/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=5, models=Ridge(), models__alpha=0.001;, score=0.385 total time=   0.1s
[CV 5/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=5, models=Ridge(), models__alpha=0.001;, score=0.383 total time=   0.1s
[CV 1/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=5, models=Ridge(), models__alpha=0.01;, score=0.419 total time=   0.1s
[CV 2/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=5, models=Ridge(), models__alpha=0.01;, score=0.366 total time=   0.1s
[CV 3/5] END features__pipeline1__selection__k=3, features__pipeline2__pca__n_components=5, models=Ridge(), models__alpha=0.01;, score=0.380 total time=   0.1s
[CV 4/5] END features__pipeline1__sel

[CV 5/5] END features__pipeline1__selection__k=4, features__pipeline2__pca__n_components=4, models=Ridge(), models__alpha=0.001;, score=0.378 total time=   0.1s
[CV 1/5] END features__pipeline1__selection__k=4, features__pipeline2__pca__n_components=4, models=Ridge(), models__alpha=0.01;, score=0.413 total time=   0.1s
[CV 2/5] END features__pipeline1__selection__k=4, features__pipeline2__pca__n_components=4, models=Ridge(), models__alpha=0.01;, score=0.365 total time=   0.1s
[CV 3/5] END features__pipeline1__selection__k=4, features__pipeline2__pca__n_components=4, models=Ridge(), models__alpha=0.01;, score=0.379 total time=   0.1s
[CV 4/5] END features__pipeline1__selection__k=4, features__pipeline2__pca__n_components=4, models=Ridge(), models__alpha=0.01;, score=0.380 total time=   0.1s
[CV 5/5] END features__pipeline1__selection__k=4, features__pipeline2__pca__n_components=4, models=Ridge(), models__alpha=0.01;, score=0.378 total time=   0.1s
[CV 1/5] END features__pipeline1__selec

C:\Users\bevli\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "C:\Users\bevli\anaconda3\lib\site-packages\joblib\memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File 

[CV 1/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=10;, score=nan total time=   0.0s
[CV 2/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=10;, score=nan total time=   0.0s
[CV 3/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=10;, score=nan total time=   0.0s
[CV 4/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=10;, score=nan total time=   0.0s
[CV 5/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=3, models=Ridge(), models__alpha=10;, score=nan total time=   0.0s
[CV 1/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=4, models=Ridge(), models__alpha=0.001;, score=nan total time=   0.0s
[CV 2/5] END features__pipeline1__selection__k=5, features__p

...

[CV 1/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.2, models__max_depth=10, models__n_estimators=150;, score=nan total time=   0.0s
[CV 2/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.2, models__max_depth=10, models__n_estimators=150;, score=nan total time=   0.0s
[CV 3/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.2, models__max_depth=10, models__n_estimators=150;, score=nan total time=   0.0s
[CV 4/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.2, models__max_depth=10, models__n_estimators=150;, score=nan total time=   0.0s
[CV 5/5] END features__pipeline1__selection__k=5, features__pipeline

C:\Users\bevli\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "C:\Users\bevli\anaconda3\lib\site-packages\joblib\memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File 

[CV 4/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.3, models__max_depth=5, models__n_estimators=200;, score=nan total time=   0.0s
[CV 5/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.3, models__max_depth=5, models__n_estimators=200;, score=nan total time=   0.0s
[CV 1/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.3, models__max_depth=7, models__n_estimators=100;, score=nan total time=   0.0s
[CV 2/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.3, models__max_depth=7, models__n_estimators=100;, score=nan total time=   0.0s
[CV 3/5] END features__pipeline1__selection__k=5, features__pipeline2__p

C:\Users\bevli\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "C:\Users\bevli\anaconda3\lib\site-packages\joblib\memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "C:\Users\bevli\anaconda3\lib\site-packages\sklearn\pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File 

[CV 2/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.3, models__max_depth=10, models__n_estimators=150;, score=nan total time=   0.0s
[CV 3/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.3, models__max_depth=10, models__n_estimators=150;, score=nan total time=   0.0s
[CV 4/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.3, models__max_depth=10, models__n_estimators=150;, score=nan total time=   0.0s
[CV 5/5] END features__pipeline1__selection__k=5, features__pipeline2__pca__n_components=5, models=GradientBoostingRegressor(), models__learning_rate=0.3, models__max_depth=10, models__n_estimators=150;, score=nan total time=   0.0s
[CV 1/5] END features__pipeline1__selection__k=5, features__pipeline

In [27]:
print('Final training score is: ', gridsearch.score(df_train, y_train))

Final training score is:  0.6170127100481984


In [29]:
print('Final score is: ', gridsearch.score(df_test, y_test))

Final score is:  0.5967722256702173


In [30]:
gridsearch.best_params_


{'features__pipeline1__selection__k': 4,
 'features__pipeline2__pca__n_components': 4,
 'models': RandomForestRegressor(max_depth=5, n_estimators=150),
 'models__max_depth': 5,
 'models__n_estimators': 150}